In [1]:
import pandas as pd

# Loading the data set

In [2]:

data = pd.read_csv('./data/train.csv')
data = data.drop(columns=['time'])
# Compute split index
split_idx = int(0.9 * len(data))

# Split
df_train = data.iloc[:split_idx]
df_evaluation  = data.iloc[split_idx:]
data.head()

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,Y1,Y2
0,0.207366,-0.159951,-0.634176,-0.580962,-0.266505,0.060173,-0.475257,-1.486516,-0.332594,-0.671466,-0.226149,-0.187624,-0.780237,-0.785965,-0.935902,-0.310081
1,0.188828,-0.265508,0.042143,-0.550442,-0.132319,-0.185219,0.028295,0.093210,-0.518139,-0.251917,-0.347845,-0.359069,-0.161254,0.020401,-0.089707,-0.305374
2,-0.144261,-0.577142,-0.214634,-0.747391,-0.184255,-0.464831,-0.085181,0.700449,-0.603438,0.197773,-0.566696,-0.580799,0.202726,0.135261,-0.077855,-0.631485
3,0.208982,-0.310449,0.513708,-0.562868,0.742308,-0.305487,0.762246,1.363020,-0.384575,0.525556,-0.348514,-0.428099,0.548993,0.471031,0.941271,-0.535212
4,0.093320,-0.358156,0.173188,-0.687296,-0.161461,-0.116062,-0.245748,0.863372,-0.655588,-0.263358,-0.557428,-0.481214,0.083602,0.003087,-0.039582,-0.490561


# Training

In [3]:
import torch
import torch.nn as nn

class LSTMDecoderOnly(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=1):
        """
        input_dim : dimension of known part of vector + predicted missing part (during training)
        hidden_dim: LSTM hidden size
        output_dim: dimension of missing part to predict
        """
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, known_future, missing_future=None, teacher_forcing=True):
        """
        known_future : (batch, horizon, known_dim)
        missing_future : (batch, horizon, missing_dim), ground truth (optional, for teacher forcing)
        teacher_forcing : if True, feed ground truth missing part at each step during training
        """
        batch_size, horizon, known_dim = known_future.shape
        missing_dim = self.fc.out_features

        # Initialize LSTM hidden state
        h = torch.zeros(1, batch_size, self.lstm.hidden_size, device=known_future.device)
        c = torch.zeros(1, batch_size, self.lstm.hidden_size, device=known_future.device)

        outputs = []
        prev_missing = torch.zeros(batch_size, missing_dim, device=known_future.device)

        for t in range(1, horizon):
            # Get known part for this step
            known_t = known_future[:, t, :]   # (batch, known_dim)

            # Choose missing part input
            if teacher_forcing :
                if missing_future is not None:
                    missing_input = missing_future[:, t-1, :]
                else:
                    missing_input = prev_missing
            else:
                if missing_future is not None and not torch.isnan(missing_future[:, t-1, :]).any():
                    missing_input = missing_future[:, t-1, :]
                else:
                    missing_input = prev_missing

            # Concatenate known and missing input
            decoder_input = torch.cat([known_t, missing_input], dim=-1).unsqueeze(1)  # (batch, 1, input_dim)

            # Run through LSTM
            out, (h, c) = self.lstm(decoder_input, (h, c))

            # Predict missing part
            pred_missing = self.fc(out.squeeze(1))  # (batch, missing_dim)

            outputs.append(pred_missing)
            prev_missing = pred_missing.detach()  # for autoregression

        outputs = torch.stack(outputs, dim=1)  # (batch, horizon, missing_dim)
        return outputs


In [4]:
from torch.utils.data import DataLoader, TensorDataset, random_split, Subset
import torch

# Parameters
horizon = 1000
known_dim = 14
missing_dim = 2

# Example: df_train with T rows (timesteps) and (known_dim + missing_dim) columns
# Let's say df_train.shape = (T, 20)
T = len(df_train)

# Collect windows
X_known_list, X_missing_list = [], []

step = 100  # non-overlapping windows
for i in range(0, T - horizon + 1, step):
    window = df_train.iloc[i:i+horizon].values  # shape (horizon, known_dim+missing_dim)
    
    X_known_list.append(window[:, :known_dim])                   # first half
    X_missing_list.append(window[:, known_dim:known_dim+missing_dim])  # second half

# Convert to tensors
X_known = torch.tensor(X_known_list, dtype=torch.float32)     # (N, horizon, known_dim)
X_missing = torch.tensor(X_missing_list, dtype=torch.float32) # (N, horizon, missing_dim)

print("X_known:", X_known.shape)
print("X_missing:", X_missing.shape)

# Wrap in dataset
dataset = TensorDataset(X_known, X_missing)

# Split 80/20
N = len(dataset)                    # total number of windows
train_size = int(0.9 * N)           # 80% for training
val_size = N - train_size           # 20% for validation

# Train = first 80%, Val = last 20%
train_ds = Subset(dataset, range(0, train_size))
val_ds   = Subset(dataset, range(train_size, N))

# DataLoaders
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=32)


X_known: torch.Size([711, 1000, 14])
X_missing: torch.Size([711, 1000, 2])


C:\Users\adele\AppData\Local\Temp\ipykernel_26956\1177528080.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:256.)
  X_known = torch.tensor(X_known_list, dtype=torch.float32)     # (N, horizon, known_dim)


In [5]:
import torch.nn as nn
import torch.optim as optim

# Model
hidden_dim = 32
model = LSTMDecoderOnly(input_dim=known_dim + missing_dim,
                        hidden_dim=hidden_dim,
                        output_dim=missing_dim)

criterion = nn.MSELoss()   # or MAE, depending on your task
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
num_epochs = 20
train_losses = []
val_losses = []
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for xb_known, xb_missing in train_loader:
        optimizer.zero_grad()
        pred = model(xb_known, xb_missing, teacher_forcing=True)  # training
        loss = criterion(pred, xb_missing[:, 1:, :])  # align target to output shape
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * xb_known.size(0)

    train_loss /= len(train_loader.dataset)
    train_losses.append(train_loss)

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for xb_known, xb_missing in val_loader:
            pred = model(xb_known, teacher_forcing=False)  # autoregressive inference
            loss = criterion(pred, xb_missing[:, 1:, :])  # align target to output shape
            val_loss += loss.item() * xb_known.size(0)

    val_loss /= len(val_loader.dataset)
    val_losses.append(val_loss)

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

Epoch 1/20 - Train Loss: 0.6965, Val Loss: 0.9495
Epoch 2/20 - Train Loss: 0.5683, Val Loss: 0.7705
Epoch 3/20 - Train Loss: 0.4716, Val Loss: 0.6482
Epoch 4/20 - Train Loss: 0.3832, Val Loss: 0.5422
Epoch 5/20 - Train Loss: 0.3249, Val Loss: 0.5002
Epoch 6/20 - Train Loss: 0.2957, Val Loss: 0.4793
Epoch 7/20 - Train Loss: 0.2740, Val Loss: 0.4676
Epoch 8/20 - Train Loss: 0.2570, Val Loss: 0.4558
Epoch 9/20 - Train Loss: 0.2432, Val Loss: 0.4375
Epoch 10/20 - Train Loss: 0.2323, Val Loss: 0.4224
Epoch 11/20 - Train Loss: 0.2237, Val Loss: 0.4202
Epoch 12/20 - Train Loss: 0.2163, Val Loss: 0.4140
Epoch 13/20 - Train Loss: 0.2101, Val Loss: 0.4051
Epoch 14/20 - Train Loss: 0.2045, Val Loss: 0.4079
Epoch 15/20 - Train Loss: 0.1998, Val Loss: 0.3972
Epoch 16/20 - Train Loss: 0.1956, Val Loss: 0.3907
Epoch 17/20 - Train Loss: 0.1917, Val Loss: 0.3909
Epoch 18/20 - Train Loss: 0.1880, Val Loss: 0.3931
Epoch 19/20 - Train Loss: 0.1847, Val Loss: 0.3953
Epoch 20/20 - Train Loss: 0.1817, Val Lo

In [6]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    y=train_losses,
    mode="lines+markers",
    name="Train Loss"
))

fig.add_trace(go.Scatter(
    y=val_losses,
    mode="lines+markers",
    name="Validation Loss"
))

fig.update_layout(
    title="Training vs Validation Loss",
    xaxis_title="Epoch",
    yaxis_title="Loss",
    template="plotly_white",
    legend=dict(x=0, y=1)
)

fig.show()

# Evaluation

In [16]:

X_known = torch.tensor(data.values[:, :known_dim], dtype=torch.float32).unsqueeze(0)     # (N, horizon, known_dim)
X_missing = torch.tensor(data.values[:, known_dim:known_dim+missing_dim], dtype=torch.float32) # (N, horizon, missing_dim)
X_missing_prepared = X_missing.clone()
X_missing = X_missing.unsqueeze(0)
X_missing_prepared[df_train.shape[0]:] = float('nan')
X_missing_prepared = X_missing_prepared.unsqueeze(0)  # Mask evaluation part

In [8]:
model.eval()
with torch.no_grad():
    pred_missing = model(X_known, X_missing_prepared, teacher_forcing=False)

print(pred_missing.shape)  # (1, horizon, missing_dim)

torch.Size([1, 79999, 2])


In [9]:
loss = criterion(pred_missing[:,df_train.shape[0]:,:], X_missing[:, df_train.shape[0]+1:, :])  # align target to output shape
print("Evaluation Loss on full data:", loss.item())

Evaluation Loss on full data: 0.9531955718994141


In [18]:
import plotly.graph_objects as go

# Extract arrays for feature 0
pred = pred_missing[0, df_train.shape[0]:, 0].cpu().numpy()   # (horizon,)
gt   = X_missing[0, 1+df_train.shape[0]:, 0].cpu().numpy()     # (horizon-1,)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(range(len(gt))),
    y=gt,
    mode="lines",
    name="Y1 Ground truth"
))

# Prediction
fig.add_trace(go.Scatter(
    x=list(range(len(pred))),
    y=pred,
    mode="lines+markers",
    name="Y1 Prediction"
))

fig.update_layout(
    title="Y1 Prediction vs Ground Truth",
    xaxis_title="Time step",
    yaxis_title="Value",
    template="plotly_white"
)

fig.show()



# Extract arrays for feature 0
pred = pred_missing[0, df_train.shape[0]:, 1].cpu().numpy()   # (horizon,)
gt   = X_missing[0, 1+df_train.shape[0]:, 1].cpu().numpy()     # (horizon-1,)

fig = go.Figure()

# Ground truth (shifted by 1)
fig.add_trace(go.Scatter(
    x=list(range(len(gt))),
    y=gt,
    mode="lines",
    name="Y2 Ground truth"
))

# Prediction
fig.add_trace(go.Scatter(
    x=list(range(len(pred))),
    y=pred,
    mode="lines+markers",
    name="Y2 Prediction"
))

fig.update_layout(
    title="Y2 Prediction vs Ground Truth",
    xaxis_title="Time step",
    yaxis_title="Value",
    template="plotly_white"
)

fig.show()


# Final Inference Submission

In [19]:
from torch.utils.data import DataLoader, TensorDataset, random_split, Subset
import torch

T = len(data)

# Collect windows
X_known_list, X_missing_list = [], []

for i in range(0, T - horizon + 1, step):
    window = data.iloc[i:i+horizon].values  # shape (horizon, known_dim+missing_dim)
    
    X_known_list.append(window[:, :known_dim])                   # first half
    X_missing_list.append(window[:, known_dim:known_dim+missing_dim])  # second half

# Convert to tensors
X_known = torch.tensor(X_known_list, dtype=torch.float32)     # (N, horizon, known_dim)
X_missing = torch.tensor(X_missing_list, dtype=torch.float32) # (N, horizon, missing_dim)

print("X_known:", X_known.shape)
print("X_missing:", X_missing.shape)

# Wrap in dataset
dataset = TensorDataset(X_known, X_missing)

# Split 80/20
N = len(dataset)                    # total number of windows
train_size = int(1.0 * N)           # 80% for training
val_size = N - train_size           # 20% for validation

# Train = first 80%, Val = last 20%
train_ds = Subset(dataset, range(0, train_size))
val_ds   = Subset(dataset, range(train_size, N))

# DataLoaders
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=32)


X_known: torch.Size([791, 1000, 14])
X_missing: torch.Size([791, 1000, 2])


In [20]:
import torch.nn as nn
import torch.optim as optim

# Model
model = LSTMDecoderOnly(input_dim=known_dim + missing_dim,
                        hidden_dim=hidden_dim,
                        output_dim=missing_dim)

criterion = nn.MSELoss()   # or MAE, depending on your task
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
train_losses = []
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for xb_known, xb_missing in train_loader:
        optimizer.zero_grad()
        pred = model(xb_known, xb_missing, teacher_forcing=True)  # training
        loss = criterion(pred, xb_missing[:, 1:, :])  # align target to output shape
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * xb_known.size(0)

    train_loss /= len(train_loader.dataset)
    train_losses.append(train_loss)

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f}")

Epoch 1/20 - Train Loss: 0.8126
Epoch 2/20 - Train Loss: 0.6249
Epoch 3/20 - Train Loss: 0.4931
Epoch 4/20 - Train Loss: 0.4114
Epoch 5/20 - Train Loss: 0.3704
Epoch 6/20 - Train Loss: 0.3402
Epoch 7/20 - Train Loss: 0.3157
Epoch 8/20 - Train Loss: 0.2979
Epoch 9/20 - Train Loss: 0.2848
Epoch 10/20 - Train Loss: 0.2745
Epoch 11/20 - Train Loss: 0.2658
Epoch 12/20 - Train Loss: 0.2587
Epoch 13/20 - Train Loss: 0.2522
Epoch 14/20 - Train Loss: 0.2462
Epoch 15/20 - Train Loss: 0.2409
Epoch 16/20 - Train Loss: 0.2361
Epoch 17/20 - Train Loss: 0.2315
Epoch 18/20 - Train Loss: 0.2276
Epoch 19/20 - Train Loss: 0.2239
Epoch 20/20 - Train Loss: 0.2207


In [21]:
import numpy as np
data_final = pd.read_csv('./data/test.csv')
data_final = data_final.drop(columns=['time'])
data_final["Y1"] = np.nan
data_final["Y2"] = np.nan

data_final = pd.concat([data, data_final], axis=0).reset_index(drop=True)

In [22]:
X_known = torch.tensor(data_final.values[:, :known_dim], dtype=torch.float32).unsqueeze(0)     # (N, horizon, known_dim)
X_missing = torch.tensor(data_final.values[:, known_dim:known_dim+missing_dim], dtype=torch.float32).unsqueeze(0) # (N, horizon, missing_dim)

In [23]:
model.eval()
with torch.no_grad():
    pred_missing = model(X_known, X_missing, teacher_forcing=False)

print(pred_missing.shape)  # (1, horizon, missing_dim)

torch.Size([1, 95995, 2])


In [24]:
pred_array = pred_missing.squeeze(0).cpu().numpy()  # (horizon, missing_dim)

# Step 2: create DataFrame
col_names = [f"Y{i}" for i in range(pred_array.shape[1])]
df_pred = pd.DataFrame(pred_array, columns=col_names)

# Step 3: save to CSV
df_pred.to_csv("final_inference_submission.csv", index=False)